In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv(r'C:\Users\Emal John Manuel\Desktop\MLops-new\notebooks\data.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
235,I know that many people will/have automaticall...,negative
437,I want to state first that I am a Christian (a...,negative
102,It's interesting how the train of research can...,positive
421,"Ringmaster, Jerry Springer's pathetic excuse f...",negative
29,"I went to see Random Hearts with 3 friends, an...",negative


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to C:\Users\Emal John
[nltk_data]     Manuel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')  # For WordNet lemmatization support


[nltk_data] Downloading package stopwords to C:\Users\Emal John
[nltk_data]     Manuel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Emal John
[nltk_data]     Manuel\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to C:\Users\Emal John
[nltk_data]     Manuel\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package omw-1.4 to C:\Users\Emal John
[nltk_data]     Manuel\AppData\Roaming\nltk_data...


True

In [7]:
df = normalize_text(df)
df.head()

,review,sentiment
235,know many people automatically given film rati...,negative
437,want state first christian work film tv indust...,negative
102,interesting train research flow started lookin...,positive
421,ringmaster jerry springer pathetic excuse wast...,negative
29,went see random heart friend first thought may...,negative


In [8]:
df['sentiment'].value_counts()

sentiment
negative    269
positive    231
Name: count, dtype: int64

In [9]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [10]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
235,know many people automatically given film rati...,0
437,want state first christian work film tv indust...,0
102,interesting train research flow started lookin...,1
421,ringmaster jerry springer pathetic excuse wast...,0
29,went see random heart friend first thought may...,0


In [11]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [12]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [15]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/emaljm/MLops-new.mlflow')
dagshub.init(repo_owner='emaljm', repo_name='MLops-new', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=43379475-fd48-4294-9b03-2d5bc3c75789&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=fabcbe06fdf00143bdf88f7319fa8b15eb9ac9e109a3c85423e40d06a0f6aa28




Accessing as emaljm

Initialized MLflow to track repo "emaljm/MLops-new"

Repository emaljm/MLops-new initialized!

2025/10/20 15:38:20 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/0dd85957b6d14170b80205484ebc2098', creation_time=1760954900852, experiment_id='0', last_update_time=1760954900852, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [16]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-10-20 15:39:04,185 - INFO - Starting MLflow run...
2025-10-20 15:39:04,893 - INFO - Logging preprocessing parameters...
2025-10-20 15:39:05,955 - INFO - Initializing Logistic Regression model...
2025-10-20 15:39:05,955 - INFO - Fitting the model...
2025-10-20 15:39:06,002 - INFO - Model training complete.
2025-10-20 15:39:06,002 - INFO - Logging model parameters...
2025-10-20 15:39:06,340 - INFO - Making predictions...
2025-10-20 15:39:06,342 - INFO - Calculating evaluation metrics...
2025-10-20 15:39:06,342 - INFO - Logging evaluation metrics...
2025-10-20 15:39:07,668 - INFO - Saving and logging the model...
2025/10/20 15:39:23 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025-10-20 15:39:23,058 - INFO - Model training and logging completed in 18.17 seconds.
2025-10-20 15:39:23,061 - INFO - Accuracy: 0.68
2025-10-20 15:39:23,062 - INFO - Precision: 0.6166666666666667
2025-10-